In [1]:
#test jupyternotebook- now all functions EXCEPT classify_das.py can be run as individual functions
from DAS_extract_V2 import grab_PMIDs_from_csv
from DAS_extract_V2 import isolate_rows_by_PMID
from DAS_extract_V2 import extract_files
from DAS_extract_V2 import get_nxml
from DAS_extract_V2 import citation_count
import pandas as pd
import numpy as np

#a test to see if it works in jupyter notebook
#file = "example_files/PMC_parser_output.csv"
#df1 = pd.read_csv(file)
#df2 = df1['DOI']
#df1['citation_count'] = citation_count(df2)
#print(df1)

In [ ]:
# %run /path/to/my_script.py FOR CLASSIFY DAS python file 
# !python /path/to/my_script.py

In [6]:
#STEP 1 -grab PMIDs from CSHL digital repo CSV 
from DAS_extract_V2 import grab_PMIDs_from_csv
from DAS_extract_V2 import isolate_rows_by_PMID
from DAS_extract_V2 import download_PMC
from DAS_extract_V2 import extract_files
from DAS_extract_V2 import get_nxml

#datafile = "example_files/2023_07_06_CSHL_articles_2007-2022_from_IR.csv"
#pmids_output_file = "example_files/pmids_07_22.txt"

#grab_PMIDs_from_csv(datafile,pmids_output_file)

#STEP2 check OA database CSV for files to download-oa_file_list was not able to be uploaded to Github, you must download your own copy from
#go to the Base FTP URL https://ftp.ncbi.nlm.nih.gov/pub/pmc/
#isolate_rows_by_PMID('example_files/pmids_07_22.txt', 'example_files/oa_file_list.csv', 'example_files/output1_07_22_PMC.csv')

#STEP 3 - using the links in the output file and download zipped PMC folders containing .nxml, .pdf, figures and data into a directory (download_PMC fxn)
#STEP 3A- Each downladed file will look like PMC{ID Number}.tar.gz, using extract_files function to unzip them 

#download_PMC('example_files/output1_07_22_PMC.csv', "example_files/PMC_folder_07_22")
#folder = 'example_files/PMC_folder_07_22'
#extract_files(folder)

#STEP 4 grab the full text XML from each PMC folder and create a folder with all articles
source_folder = 'example_files/PMC_folder_07_22'
target_folder = 'example_files/final_xml_07_22'
file_extension = '.nxml'

get_nxml(source_folder,target_folder,file_extension)

In [4]:
from DAS_extract_V2 import parse_xml_directory
#IN THIS STEP WE ARE PARSING THROUGH EACH FULL TEXT XML and returning key article metadata including DAS and url links

xml_directory = '/Users/muthuku/Desktop/final_xml_07_22'
df1 = parse_xml_directory(xml_directory)
#DATAFRAME cleanup steps - run these after the main function above
# create a combined DAS column to include results from all methods of DAS collection 
df1['combined_DAS'] = df1['Data Availability 1'] + df1['Data Availability 2'] + df1['Data Availability 3'] + df1['Data Availability 4'] + df1['Data Availability 5'] 
df1['unique_combined_das'] = df1['combined_DAS'].apply(lambda x: list(set(x)))
count = 0
for n in df1['unique_combined_das']:
	count += 1
	if n == []:
		n.append("Not applicable " + str(count))

#combine URLs as well 
df1['all_urls'] = df1['URL1'] +df1['URL2'] + df1['URL3'] +df1['URL4'] + df1['URL5']
#drop individual columns 
column_list = ['Data Availability 1','Data Availability 2','Data Availability 3','Data Availability 4','Data Availability 5','URL1', 'URL2','URL3','URL4','URL5', 'combined_DAS']
df2= df1.drop(columns = column_list)
#convert the list of seperate strings into a single string- join all sentences together and remove commas which interfere with ML 
df2['combined_string_DAS'] = df2['unique_combined_das'].apply(lambda x: ' '.join(x))
df2['combined_string_DAS'] = df2['combined_string_DAS'].str.replace(',', '')
df2 = df2.drop(['unique_combined_das'], axis = 1)
df2.to_csv("example_files/PMC_parser_output_test.csv", encoding = "utf-8")

/Users/muthuku/Desktop/final_xml_07_22/0060580.nxml
/Users/muthuku/Desktop/final_xml_07_22/10.1177_1177271917695810.nxml
/Users/muthuku/Desktop/final_xml_07_22/10.1177_1177271917702895.nxml
/Users/muthuku/Desktop/final_xml_07_22/10.1177_2472555218766842.nxml
/Users/muthuku/Desktop/final_xml_07_22/10020_2021_Article_307.nxml
/Users/muthuku/Desktop/final_xml_07_22/10020_2022_Article_483.nxml
/Users/muthuku/Desktop/final_xml_07_22/1031.nxml
/Users/muthuku/Desktop/final_xml_07_22/1032.nxml
/Users/muthuku/Desktop/final_xml_07_22/10456_2020_Article_9727.nxml
/Users/muthuku/Desktop/final_xml_07_22/1064.nxml
/Users/muthuku/Desktop/final_xml_07_22/10911_2021_Article_9486.nxml
/Users/muthuku/Desktop/final_xml_07_22/11032_2009_Article_9327.nxml
/Users/muthuku/Desktop/final_xml_07_22/1126.nxml
/Users/muthuku/Desktop/final_xml_07_22/11307_2014_Article_741.nxml
/Users/muthuku/Desktop/final_xml_07_22/11693_2009_Article_9022.nxml
/Users/muthuku/Desktop/final_xml_07_22/1195.nxml
/Users/muthuku/Desktop/

In [12]:
#%run ./DAS_extract_V2/classify_das2.py
#RUN THE ML CLASSIFER IN THE 
#you also need this large file glove.6B.50d.txt, which is necessary for the
#vectorizing of words in the DAS
#!python3 /path/to/my_script.py --istest no --user_input yes --combine_labels yes --coding 2 --stopwords no --uniform_prior yes --stemming no --skip_model1 yes --skip_model2 no --skip_model3 yes --skip_model4 no --skip_model5 yes
!python3 ./DAS_extract_V2/classify_das2.py --istest no --user_input yes --combine_labels no --coding 1 --stopwords no --uniform_prior yes --stemming yes --skip_model1 yes --skip_model2 yes --skip_model3 no --skip_model4 yes --skip_model5 yes

length_train: 140
length_test: 35
140 random_indices_train
35 random_indices_test
Reading input data...
max_number:49
48 data points to classify
Combine labels: no coding: 1 stopwords: no stemming: yes uniform prior: yes
Reading annotated data...
Max_number annotated:177
Test labels: {'5', '0', '2', '1', '4', '3'}
Train labels: {'5', '0', '1', '2', '4', '3'}
Training data points:
140 training data points
Test data points:
34 test data points
Top-frequency test data points:
34 top-frequency test data points
Training TF-IDF SVM classifier...
Testing TF-IDF SVM classifier...
0.7352941176470589
0.7352941176470589
Weighted accuracy: 0.7352941176470589
Grid Search for SVM classifier...
Classifying statements...
Printing classified statements...
Printing classified test set...
There are 34 test texts and  34 predicted test texts


In [18]:
#%run ./DAS_extract_V2/classify_das2.py
#RUN THE ML CLASSIFER IN THE 
#you also need this large file glove.6B.50d.txt, which is necessary for the
#vectorizing of words in the DAS
#!python3 /path/to/my_script.py --istest no --user_input yes --combine_labels yes --coding 2 --stopwords no --uniform_prior yes --stemming no --skip_model1 yes --skip_model2 no --skip_model3 yes --skip_model4 no --skip_model5 yes
!python3 ./DAS_extract_V2/classify_das2.py --istest no --user_input yes --combine_labels no --coding 1 --stopwords no --uniform_prior yes --stemming yes --skip_model1 yes --skip_model2 yes --skip_model3 no --skip_model4 yes --skip_model5 yes

length_train: 140
length_test: 35
140 random_indices_train
35 random_indices_test
Reading input data...
max_number:49
48 data points to classify
Combine labels: no coding: 1 stopwords: no stemming: yes uniform prior: yes
Reading annotated data...
Max_number annotated:177
Test labels: {'5', '4', '2', '3', '0', '1'}
Train labels: {'4', '5', '2', '3', '0', '1'}
Training data points:
140 training data points
Test data points:
34 test data points
Top-frequency test data points:
34 top-frequency test data points
Training TF-IDF SVM classifier...
Testing TF-IDF SVM classifier...
0.6764705882352942
0.6764705882352942
Weighted accuracy: 0.6764705882352942
Grid Search for SVM classifier...
Classifying statements...
Printing classified statements...
Printing classified test set...
There are 34 test texts and  34 predicted test texts


In [19]:
from DAS_extract_V2 import merge_og_classify_dfs
from DAS_extract_V2 import citation_count

file1 = "example_files/PMC_parser_output.csv"
file2 = "example_files/ML_output/Classified_SVM_combined_labels_no-coding-approach1-stopwords-no-uniformprior_yes-stemming_yes-test_no.csv"
df3 = merge_og_classify_dfs(file1, file2)

df4 = df3["DOI"]
df3['citation_count'] = citation_count(df4)
#code to get average citation counts and class count number by year 

# result = df3.groupby(['Year', 'classifying_number']).size().reset_index(name='counts')
# average_citation_by_class_year = df3.groupby(['Year', 'classifying_number'])['citation_count'].mean().reset_index()
#TO GET JOURNAL COUNTS 
#journal_counts = df3['Journal'].value_counts().reset_index()
#journal_counts.columns = ['Journal', 'Count']

#print(journal_counts)
df3.to_csv("example_files/final_output_test_JN.csv", encoding = "utf-8", index = False)

#STEP4 -citation counts for each article using SCOPUS API and using article DOIs as an input -TEST



#file = "example_files/PMC_parser_output.csv"
#df1 = pd.read_csv(file)
#df2 = df1['DOI']
#df1['citation_count'] = citation_count(df2)
#print(df1)

#df1.to_csv('og_df_with_citations1.csv', encoding = "utf-8", index = False

100%|███████████████████████████████████████| 1318/1318 [04:34<00:00,  4.81it/s]
